In [1]:
import sys
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)

In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch

connCredential = {}
connCredential['host'] = "http://47.91.72.40"
connCredential['port'] = "9200"
connCredential['conn'] = [connCredential['host']+":"+connCredential['port']]
connCredential['http_auth']=('elastic', 'Couronne3...')

es = Elasticsearch(connCredential['conn'], http_auth=('elastic', 'Couronne3...'))

In [3]:
from yelpquery import YelpQuery
ye = YelpQuery()

In [4]:
import re
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

def find_n_neighbours(df,n,nrows):
    if n>nrows:
        n = nrows
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

def recommend(userID): #Op-cYCn71IJiIfx1IVjAwA
    #### get friends
    res = ye.getSingleMatch(index="yelp-user*", field="user_id", value=userID, size=1)
    friend_string = res['hits']['hits'][0]["_source"]['friends']
    users = re.findall(r',?([\-\_\w]+),?', friend_string)
    users.append(userID)
    #### get friends reviews
    firstReview = ye.getMultivalueTerm(index="yelp-review*", term="user_id.keyword", valueArray=users, size=100,exclude=["funny","cool","text","date","useful"])
    df_review = ye.getResultScrolling(firstReview)
    #### get rated businesses info
    restaurants = df_review.business_id.unique()
    firstBiz = ye.getMultivalueTerm(index="yelp-business*", term="business_id.keyword", valueArray=restaurants, size=100)
    df_biz = ye.getResultScrolling(firstBiz)
    df_biz = df_biz[["business_id","city","categories","is_open"]]
    ### gather everything
    df = pd.merge(df_review,df_biz,how="inner", on='business_id')
    df = df[["user_id", "business_id","stars","city","categories","is_open"]]
    ### normalisation step for the ratings
    Mean = df.groupby(by="user_id",as_index=False)['stars'].mean()
    Mean.rename(columns={'stars': 'mean'}, inplace=True)
    Rating_avg = pd.merge(df,Mean,on='user_id')
    Rating_avg['adg_rating']=Rating_avg['stars']-Rating_avg['mean']
    ### keep track of which user rated what
    check = pd.pivot_table(Rating_avg,values='stars',index='user_id',columns='business_id')
    ### create matrix
    final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='business_id')
    nrows = final.shape[0]
    # Replacing NaN by business Average
    final_business = final.fillna(final.mean(axis=0))
    # user similarity on replacing NAN by item(business) avg
    cosine = cosine_similarity(final_business)
    np.fill_diagonal(cosine, 0 )
    similarity_with_businesses = pd.DataFrame(cosine,index=final_business.index)
    similarity_with_businesses.columns=final_business.index
    similarity_with_businesses.head()
    # top 30 neighbours for each user
    sim_user_30_m = find_n_neighbours(similarity_with_businesses,30,nrows)
    ### _b
    #Rating_avg = Rating_avg.astype({"business_id": str})
    business_user = Rating_avg.groupby(by = 'user_id')['business_id'].apply(lambda x:','.join(x))
    predicted_businesses = user_item_score(userID,check,sim_user_30_m,business_user,final_business,Mean,similarity_with_businesses,df_biz)
    return predicted_businesses

def user_item_score(user,check,sim_user_30_m,business_user,final_business,Mean,similarity_with_businesses,df_biz):
    businesses_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = business_user[business_user.index.isin(b)]
    l = ','.join(d.values)
    businesses_seen_by_similar_users = l.split(',')
    businesses_under_consideration = list(set(businesses_seen_by_similar_users)-set(list(map(str, businesses_seen_by_user))))
    #businesses_under_consideration = list(map(int, businesses_under_consideration))
    score = []
    for item in businesses_under_consideration:
        c = final_business.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'mean'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_businesses.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'business_id':businesses_under_consideration,'score':score})
    top_recommendation = data.sort_values(by='score',ascending=False)
    businesses_info = top_recommendation.merge(df_biz, how='inner', on='business_id')
    businesses_info = businesses_info[businesses_info['is_open'] == 1]
    return businesses_info

In [ ]:
recommend("AyjqBovADgbskmLrIBOMlQ") #Op-cYCn71IJiIfx1IVjAwA

In [5]:
import re
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

def user_item_score(user,check,sim_user_30_m,business_user,final_business,Mean,similarity_with_businesses,df_biz):
    businesses_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = business_user[business_user.index.isin(b)]
    l = ','.join(d.values)
    businesses_seen_by_similar_users = l.split(',')
    businesses_under_consideration = list(set(businesses_seen_by_similar_users)-set(list(map(str, businesses_seen_by_user))))
    #businesses_under_consideration = list(map(int, businesses_under_consideration))
    score = []
    for item in businesses_under_consideration:
        c = final_business.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'mean'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_businesses.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'business_id':businesses_under_consideration,'score':score})
    top_recommendation = data.sort_values(by='score',ascending=False)
    businesses_info = top_recommendation.merge(df_biz, how='inner', on='business_id')
    businesses_info = businesses_info[businesses_info['is_open'] == 1]
    return businesses_info

def find_n_neighbours(df,n,nrows):
    if n>nrows:
        n = nrows
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

def get_user_similar_restaurants( user1, user2 ):
    common_restaurants = Rating_avg[Rating_avg.user_id == user1].merge(Rating_avg[Rating_avg.user_id == user2],
    on = "business_id",
    how = "inner" )
    return common_restaurants


userID = "AyjqBovADgbskmLrIBOMlQ"

#### get friends
res = ye.getSingleMatch(index="yelp-user*", field="user_id", value=userID, size=1)
friend_string = res['hits']['hits'][0]["_source"]['friends']
users = re.findall(r',?([\-\_\w]+),?', friend_string)
users.append(userID)
#### get friends reviews
firstReview = ye.getMultivalueTerm(index="yelp-review*", term="user_id.keyword", valueArray=users, size=100,exclude=["funny","cool","text","date","useful"])
df_review = ye.getResultScrolling(firstReview)
#### get rated businesses info
restaurants = df_review.business_id.unique()
firstBiz = ye.getMultivalueTerm(index="yelp-business*", term="business_id.keyword", valueArray=restaurants, size=100)
df_biz = ye.getResultScrolling(firstBiz)
df_biz = df_biz[["business_id","city","categories","is_open"]]
### gather everything
df = pd.merge(df_review,df_biz,how="inner", on='business_id')
df = df[["user_id", "business_id","stars","city","categories","is_open"]]
### normalisation step for the ratings
Mean = df.groupby(by="user_id",as_index=False)['stars'].mean()
Mean.rename(columns={'stars': 'mean'}, inplace=True)
Rating_avg = pd.merge(df,Mean,on='user_id')
Rating_avg['adg_rating']=Rating_avg['stars']-Rating_avg['mean']
### keep track of which user rated what
check = pd.pivot_table(Rating_avg,values='stars',index='user_id',columns='business_id')
### create matrix
final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='business_id')
nrows = final.shape[0]
# Replacing NaN by business Average
final_business = final.fillna(final.mean(axis=0))
# user similarity on replacing NAN by item(business) avg
cosine = cosine_similarity(final_business)
np.fill_diagonal(cosine, 0 )
similarity_with_businesses = pd.DataFrame(cosine,index=final_business.index)
similarity_with_businesses.columns=final_business.index
# # top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_businesses,30,nrows)
sim_user_30_m
# business_user = Rating_avg.groupby(by = 'user_id')['business_id'].apply(lambda x:','.join(x))
# business_user.head()
# predicted_businesses = user_item_score(userID,check,sim_user_30_m,business_user,final_business,Mean,similarity_with_businesses,df_biz)
# predicted_businesses.score



,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
user_id,,,,,,,,,,,,,,,,,,,,,
-2CWN4JJz8ibrcRSMsEA6g,L2k9eW2NRHot8GDgYpOYDA,k_4aAlLIoOaaykQePm_m4g,oBw2I44PiK9HSAC0WvJLww,IKDpNaUUWR-_5gTu34PzMw,sgkFz8q_o6m8rjKxgtxk-w,RlGHTobZ4ysApnEoOzhVzQ,BzuMoM3QoPzEMFLgFhupBA,C0kfxY8qSfrnnRbL_YY_9A,9XQZ3rBx7IgsUZkvY-6X-w,nGz08gCSAT6M1WcYqlUNKg,...,_ZGQo0FTEFDd_iwvxmGq0w,Gz9DZ5Dwlc0Ge11z52NooQ,kC4hhF0W6tr2pb-6k5FKYQ,p0LmtgQPa6LB0i2h9MoUFQ,GtaD6BckQQfe5MXo1Mb2tg,QXHi5m8mmXgOb5fhbcwc1Q,yWCLBdZz3O7oheV-FZgqrA,2bE7tsrJB8TaBP-w1QjLdQ,Ug5raHGmCrYzdmWGoTbk0Q,FXwfrf5TeSlkpTLql2tWwQ
-C-l8EHSLXtZZVfUAUhsPA,jAT01COr30Zua1FtrZZU_Q,Gz9DZ5Dwlc0Ge11z52NooQ,IKDpNaUUWR-_5gTu34PzMw,a80SxA1KaPbY9MfjlEQlug,QXHi5m8mmXgOb5fhbcwc1Q,aV7WiC-bm5vE5llyOz7rNA,p0LmtgQPa6LB0i2h9MoUFQ,O_Z5bwC6cN5t2i7OjmThew,IJEsMXkP6axb6Y1-2NvE2A,kC4hhF0W6tr2pb-6k5FKYQ,...,Ug5raHGmCrYzdmWGoTbk0Q,kwYGKNTVv-_fngyuR-ciiQ,EWyEw-srCTlzsfN-P8vp-A,FXwfrf5TeSlkpTLql2tWwQ,nGz08gCSAT6M1WcYqlUNKg,2bE7tsrJB8TaBP-w1QjLdQ,yWCLBdZz3O7oheV-FZgqrA,BzuMoM3QoPzEMFLgFhupBA,RlGHTobZ4ysApnEoOzhVzQ,GtaD6BckQQfe5MXo1Mb2tg
-xDW3gYiYaoeVASXywTPgw,kC4hhF0W6tr2pb-6k5FKYQ,QXHi5m8mmXgOb5fhbcwc1Q,aV7WiC-bm5vE5llyOz7rNA,O_Z5bwC6cN5t2i7OjmThew,9XQZ3rBx7IgsUZkvY-6X-w,k_4aAlLIoOaaykQePm_m4g,AI3lslTlHCw2cI9v_xVCjA,sgkFz8q_o6m8rjKxgtxk-w,L2k9eW2NRHot8GDgYpOYDA,sNHleT96OPuudSJ1OniDqg,...,kwYGKNTVv-_fngyuR-ciiQ,a80SxA1KaPbY9MfjlEQlug,EWyEw-srCTlzsfN-P8vp-A,8u0HJZVuJldX-glusa_QhQ,RlGHTobZ4ysApnEoOzhVzQ,2bE7tsrJB8TaBP-w1QjLdQ,yWCLBdZz3O7oheV-FZgqrA,Ug5raHGmCrYzdmWGoTbk0Q,_ZGQo0FTEFDd_iwvxmGq0w,Rx23TGuzQy5Dvz5kh_hNAA
0Lv0apHQ4omJz8c5EQVP7A,EWyEw-srCTlzsfN-P8vp-A,kC4hhF0W6tr2pb-6k5FKYQ,8u0HJZVuJldX-glusa_QhQ,QXHi5m8mmXgOb5fhbcwc1Q,FXwfrf5TeSlkpTLql2tWwQ,O_Z5bwC6cN5t2i7OjmThew,9XQZ3rBx7IgsUZkvY-6X-w,p0LmtgQPa6LB0i2h9MoUFQ,L2k9eW2NRHot8GDgYpOYDA,AI3lslTlHCw2cI9v_xVCjA,...,_ZGQo0FTEFDd_iwvxmGq0w,xWnAwisqu_7UcDfq5gO9hQ,yWCLBdZz3O7oheV-FZgqrA,YXor8EbuC1Tc9GGwxpa7GA,kwYGKNTVv-_fngyuR-ciiQ,Ug5raHGmCrYzdmWGoTbk0Q,nGz08gCSAT6M1WcYqlUNKg,Rx23TGuzQy5Dvz5kh_hNAA,k_4aAlLIoOaaykQePm_m4g,RlGHTobZ4ysApnEoOzhVzQ
0b3FXt-EgiwMvS4scwBKeA,L2k9eW2NRHot8GDgYpOYDA,8u0HJZVuJldX-glusa_QhQ,9XQZ3rBx7IgsUZkvY-6X-w,IKDpNaUUWR-_5gTu34PzMw,IJEsMXkP6axb6Y1-2NvE2A,AI3lslTlHCw2cI9v_xVCjA,sgkFz8q_o6m8rjKxgtxk-w,Gz9DZ5Dwlc0Ge11z52NooQ,sNHleT96OPuudSJ1OniDqg,GtaD6BckQQfe5MXo1Mb2tg,...,EWyEw-srCTlzsfN-P8vp-A,QXHi5m8mmXgOb5fhbcwc1Q,RlGHTobZ4ysApnEoOzhVzQ,2bE7tsrJB8TaBP-w1QjLdQ,aV7WiC-bm5vE5llyOz7rNA,Ug5raHGmCrYzdmWGoTbk0Q,YXor8EbuC1Tc9GGwxpa7GA,a80SxA1KaPbY9MfjlEQlug,xWnAwisqu_7UcDfq5gO9hQ,Rx23TGuzQy5Dvz5kh_hNAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zUiU797KRloAzUvF1tJ_mQ,k_4aAlLIoOaaykQePm_m4g,8u0HJZVuJldX-glusa_QhQ,9XQZ3rBx7IgsUZkvY-6X-w,EWyEw-srCTlzsfN-P8vp-A,L2k9eW2NRHot8GDgYpOYDA,IKDpNaUUWR-_5gTu34PzMw,AI3lslTlHCw2cI9v_xVCjA,IJEsMXkP6axb6Y1-2NvE2A,sgkFz8q_o6m8rjKxgtxk-w,Gz9DZ5Dwlc0Ge11z52NooQ,...,nGz08gCSAT6M1WcYqlUNKg,a80SxA1KaPbY9MfjlEQlug,aV7WiC-bm5vE5llyOz7rNA,Ug5raHGmCrYzdmWGoTbk0Q,2bE7tsrJB8TaBP-w1QjLdQ,_ZGQo0FTEFDd_iwvxmGq0w,yWCLBdZz3O7oheV-FZgqrA,Rx23TGuzQy5Dvz5kh_hNAA,QXHi5m8mmXgOb5fhbcwc1Q,RlGHTobZ4ysApnEoOzhVzQ
zW3PaTqR8fN-mnoYWk6Ymg,kC4hhF0W6tr2pb-6k5FKYQ,QXHi5m8mmXgOb5fhbcwc1Q,aV7WiC-bm5vE5llyOz7rNA,O_Z5bwC6cN5t2i7OjmThew,L2k9eW2NRHot8GDgYpOYDA,9XQZ3rBx7IgsUZkvY-6X-w,IKDpNaUUWR-_5gTu34PzMw,sgkFz8q_o6m8rjKxgtxk-w,AI3lslTlHCw2cI9v_xVCjA,sNHleT96OPuudSJ1OniDqg,...,EWyEw-srCTlzsfN-P8vp-A,8u0HJZVuJldX-glusa_QhQ,kwYGKNTVv-_fngyuR-ciiQ,a80SxA1KaPbY9MfjlEQlug,yWCLBdZz3O7oheV-FZgqrA,xWnAwisqu_7UcDfq5gO9hQ,RlGHTobZ4ysApnEoOzhVzQ,_ZGQo0FTEFDd_iwvxmGq0w,YXor8EbuC1Tc9GGwxpa7GA,Rx23TGuzQy5Dvz5kh_hNAA
zYog3ZgqAtZiZECWTtlzBw,k_4aAlLIoOaaykQePm_m4g,sNHleT96OPuudSJ1OniDqg,8u0HJZVuJldX-glusa_QhQ,QXHi5m8mmXgOb5fhbcwc1Q,O_Z5bwC6cN5t2i7OjmThew,9XQZ3rBx7IgsUZkvY-6X-w,C0kfxY8qSfrnnRbL_YY_9A,BzuMoM3QoPzEMFLgFhupBA,Gz9DZ5Dwlc0Ge11z52NooQ,p0LmtgQPa6LB0i2h9MoUFQ,...,RlGHTobZ4ysApnEoOzhVzQ,a80SxA1KaPbY9MfjlEQlug,nGz08gCSAT6M1WcYqlUNKg,kC4hhF0W6tr2pb-6k5FKYQ,Rx23TGuzQ

In [7]:
a = get_user_similar_restaurants("AyjqBovADgbskmLrIBOMlQ","TprC8sujz8MkwuomrqUSiw")
#a = a.loc[ : , ['stars_x','stars_y','categories_x']]
a.head()

# 60skTN6p8SdQ2sGNKL0vRQ	bLbSNkLggFnqwNNzzq-Ijw	FQbVI3UyKlL-HfQGcRF8aA	ZCLDdUi4px9mys8vVUeIkg	TprC8sujz8MkwuomrqUSiw

,user_id_x,business_id,stars_x,city_x,categories_x,is_open_x,mean_x,adg_rating_x,user_id_y,stars_y,city_y,categories_y,is_open_y,mean_y,adg_rating_y
0,AyjqBovADgbskmLrIBOMlQ,e3PBfbtXUjMziX6XGsOMoA,5.0,Las Vegas,"[Cafes, Restaurants, Japanese, Soup, Noodles]",1,4.252294,0.747706,TprC8sujz8MkwuomrqUSiw,3.0,Las Vegas,"[Cafes, Restaurants, Japanese, Soup, Noodles]",1,3.631579,-0.631579
1,AyjqBovADgbskmLrIBOMlQ,e3PBfbtXUjMziX6XGsOMoA,5.0,Las Vegas,"[Cafes, Restaurants, Japanese, Soup, Noodles]",1,4.252294,0.747706,TprC8sujz8MkwuomrqUSiw,3.0,Las Vegas,"[Cafes, Restaurants, Japanese, Soup, Noodles]",1,3.631579,-0.631579
2,AyjqBovADgbskmLrIBOMlQ,stK7zVTmMar1ThpzT0Wjvg,4.0,Henderson,"[Restaurants, Sushi Bars, Seafood, Japanese]",1,4.252294,-0.252294,TprC8sujz8MkwuomrqUSiw,4.0,Henderson,"[Restaurants, Sushi Bars, Seafood, Japanese]",1,3.631579,0.368421
3,AyjqBovADgbskmLrIBOMlQ,stK7zVTmMar1ThpzT0Wjvg,4.0,Henderson,"[Restaurants, Sushi Bars, Seafood, Japanese]",1,4.252294,-0.252294,TprC8sujz8MkwuomrqUSiw,4.0,Henderson,"[Restaurants, Sushi Bars, Seafood, Japanese]",1,3.631579,0.368421
4,AyjqBovADgbskmLrIBOMlQ,cOJ1uIVIHCiefUyWG2wDfw,4.0,Las Vegas,"[Beer, Wine & Spirits, Restaurants, Gastropubs...",0,4.252294,-0.252294,TprC8sujz8MkwuomrqUSiw,4.0,Las Vegas,"[Beer, Wine & Spirits, Restaurants, Gastropubs...",0,3.631579,0.368421


In [30]:
def user_taste(userID):
    firstReview = ye.getSingleMatch(index="yelp-review*", field="user_id", value=userID, size=10)
    df_review = ye.getResultScrolling(firstReview)
    #### get rated businesses info
    restaurants = df_review.business_id.unique()
    firstBiz = ye.getMultivalueTerm(index="yelp-business*", term="business_id.keyword", valueArray=restaurants, size=100)
    df_biz = ye.getResultScrolling(firstBiz)
    df_biz = df_biz[["business_id","name","city","state","categories","is_open"]]
    ### gather everything
    df = pd.merge(df_review,df_biz,how="inner", on='business_id')
    df = df[["business_id","name","stars","state","city","categories","is_open","text","review_id","date"]]
    ## filter restaurants only
    df = df[df.categories.map(lambda x: 'Restaurants' in x)]
    return df.sort_values(by='stars', ascending=False)

d = user_taste("bGBp3ptS25gzxmFhVfB7WA")
d.head(20)

,business_id,name,stars,state,city,categories,is_open,text,review_id,date
45,eaNenRk_liZBERFFLCXqqQ,Hobak Korean BBQ,5.0,NV,Las Vegas,"[Barbeque, Restaurants, Korean]",1,Yum! Went here with my Korean family and trie...,-t1sqW59AvsUfPua50hiFA,2016-06-15T17:03:26.000Z
17,5WOE18ldtQ1U0MK5fN2HGQ,Rubio's Coastal Grill,5.0,NV,Las Vegas,"[Seafood, Mexican, Restaurants, Salad]",1,I love rubios. Esp on fish taco Tuesday. Since...,ZgI0jhrgeaY0QHtQcDa1HA,2018-03-16T07:43:08.000Z
50,-wCtRhzWJ40Z4F8mmg7kWg,Herringbone,5.0,NV,Las Vegas,"[Seafood, American (New), Restaurants]",1,First off.... can I get the soundtrack to this...,EYgkq_vX16LbBMQDWc1Edg,2017-08-08T05:22:43.000Z
51,tH0wF3EPPgAY14lnmK1_Sg,Makai Pacific Island Grill,5.0,NV,Las Vegas,"[Hawaiian, Barbeque, Restaurants, Asian Fusion]",1,I've heard so much hype about this place and i...,hiuGTcdwKdH1oR2vt1MJIg,2018-06-30T06:11:41.000Z
56,HfuIfYB_CxMAAJiEnxDl9w,Sushi Loca,5.0,NV,Las Vegas,"[Sushi Bars, Japanese, Restaurants]",1,My husband and I frequent here. It's our lunch...,cIz89Vf_WwvOBBGZaSv0eA,2016-11-14T06:45:17.000Z
62,Ec9CBmL3285XkeHaNp-bSQ,District One,5.0,NV,Las Vegas,"[Asian Fusion, Bars, Sports Bars, Vietnamese, ...",1,This place gets 5 stars especially due to our ...,1nN2znD695Lew7yMU5LDUg,2017-07-11T18:57:19.000Z
27,mnU3iGsL55MCWXiWV3AcAQ,In-N-Out Burger,5.0,NV,Las Vegas,"[Burgers, Fast Food, Restaurants]",1,In n out was my first hs job and even after wo...,ndD5JHi4WT5Ckr7HqnEeJA,2016-06-08T06:47:32.000Z
26,eZcCFV-8X91ZSnmB9807bw,The Cheesecake Factory,5.0,NV,Las Vegas,"[American (New), Food, Restaurants, Salad, Ame...",1,Why does this place have mediocre ratings? I ...,0Ta_DUQFUCV4usJl8-wCjQ,2016-11-14T04:55:04.000Z
41,LFs5jyYdXlzi0SpAYi1eSA,Maggiano's Little Italy,5.0,NV,Las Vegas,"[Restaurants, Italian, Bars, Nightlife]",1,I love maggionos! I always try to find a reaso...,mTAUNmUNryDIBYrhWe4viQ,2016-06-10T07:15:12.000Z
22,bRR4gsjHv0F_RnRGjQBqlg,CJ's Italian Ice & Custard,5.0,NV,Las Vegas,"[Ice Cream & Frozen Yogurt, Food, Restaurants,...",1,Never had iced custard or gelati before but my...,RCJ5_1TCoRDscfeMYYsEmg,2016-06-09T03:53:51.000Z
